In [46]:
import numpy as np
class AntColonyOptimizationTSP():
    def __init__(self, distances, n_ants, n_bests, n_iterations, decay, start=0, alpha=1, beta=1):
        self.distances = distances
        self.indexes = range(len(distances))
        self.n_ants = n_ants
        self.n_bests = n_bests
        self.n_iterations = n_iterations
        self.start = start
        self.decay = decay
        self.alpha = alpha
        self.beta = beta
        self.pheromone = np.ones(self.distances.shape) / len(distances)

    def move(self, prev, visited):
        pheromone = np.copy(self.pheromone[prev])
        pheromone[list(visited)] = 0
        distances = 1.0 / self.distances[prev]
        probabilities = pheromone**self.alpha * distances**self.beta
        probabilities /= sum(probabilities)
        return np.random.choice(self.indexes, 1, p=probabilities)[0]

    def pathGeneration(self):
        path = []
        prev = self.start
        visited = set()
        visited.add(prev)
        for _ in range(len(self.indexes) - 1):
            next = self.move(prev, visited)
            visited.add(next)
            path.append((prev, next))
            prev = next
        path.append((next, self.start))
        return path

    def pathCost(self, path):
        cost = 0
        for prev,next in path:
            cost += self.distances[prev][next]
        return cost

    def allPathsGeneration(self):
        paths = []
        for i in range(self.n_ants):
            path = self.pathGeneration()
            paths.append((path, self.pathCost(path)))
        return paths
            
    def spreadPheromone(self, paths):
        self.pheromone *= (1 - self.decay)
        for path, _ in paths:
            for prev, next in path:
                self.pheromone[prev][next] += (1 / self.distances[prev][next])

    def run(self):
        bestPath = ("None", np.inf)
        for _ in range(self.n_iterations):
            paths = sorted(self.allPathsGeneration(), key= lambda x: x[1])[:self.n_bests]
            self.spreadPheromone(paths)
            localBestPath = paths[0]
            if localBestPath[1] < bestPath[1]:
                bestPath = localBestPath
        return bestPath
        

In [50]:
distances = np.array([[np.inf, 2, 2, 5, 7],
                      [2, np.inf, 4, 8, 2],
                      [2, 4, np.inf, 1, 3],
                      [5, 8, 1, np.inf, 2],
                      [7, 2, 3, 2, np.inf]])
AntColonyOptimizationTSP(distances, 10, 6, 20, 0.5).run()

([(0, 1), (1, 4), (4, 3), (3, 2), (2, 0)], 9.0)